- 레시피를 1번에 100개씩 밖에 못가져오므로 1136개의 데이터를 모두 가져오기 위한 작업

In [7]:
import requests
import json
import time  # API 요청 사이에 잠깐 쉬기 위해 사용

# API 키 
API_KEY = "27a49415edf9496fb394"

# 전체 데이터를 저장할 리스트
all_recipes = []

# 먼저 전체 레시피 개수를 확인하기 위해 첫 요청 실행
URL = f"https://openapi.foodsafetykorea.go.kr/api/{API_KEY}/COOKRCP01/json/1/1"
response = requests.get(URL)
data = response.json()

# 전체 레시피 개수 확인
total_count = int(data["COOKRCP01"]["total_count"])
print(f"전체 레시피 개수 : {total_count}개")

전체 레시피 개수 : 1136개


In [10]:
batch_size = 100

# 1부터 total_count까지 100개씩 나눠서 요청
for start in range(1, total_count + 1, batch_size):
    end = min(start + batch_size - 1, total_count) # 마지막 범위 조정
    URL = f"https://openapi.foodsafetykorea.go.kr/api/27a49415edf9496fb394/COOKRCP01/json/{start}/{end}"
    
    print(f"요청 중 : {start} ~ {end}")
    response = requests.get(URL)
    
    if response.status_code == 200:
        data = response.json()
        recipes = data["COOKRCP01"]["row"]
        all_recipes.extend(recipes)
    else :
        print(f"API 요청 실패 : {response.status_code}")
    time.sleep(0.5) # API 서버 과부하 방지를 위해 잠깐 대기
    
# 최종 데이터 개수 출력
print(f"총 가져온 레시피 개수 : {len(all_recipes)}개")

# JSON 파일로 저정
with open("all_recipes.json", "w", encoding="utf-8") as f:
    json.dump(all_recipes, f, ensure_ascii=False, indent=4)
    
print("모든 레시피 데이터를 'all_recipes.json' 파일에 저장 완료")

요청 중 : 1 ~ 100
요청 중 : 101 ~ 200
요청 중 : 201 ~ 300
요청 중 : 301 ~ 400
요청 중 : 401 ~ 500
요청 중 : 501 ~ 600
요청 중 : 601 ~ 700
요청 중 : 701 ~ 800
요청 중 : 801 ~ 900
요청 중 : 901 ~ 1000
요청 중 : 1001 ~ 1100
요청 중 : 1101 ~ 1136
총 가져온 레시피 개수 : 1136개
모든 레시피 데이터를 'all_recipes.json' 파일에 저장 완료


In [12]:
import json
import re

# JSON 파일 로드
with open("all_recipes.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    
# 재료 목록 추출 및 전처리
recipe_ingredients = []

for recipe in data :
    if "RCP_PARTS_DTLS" in recipes :
        ingredients = recipe["RCP_PARTS_DTLS"]
        ingredients = re.sub(r"\s+", " ", ingredients) # 불필요한 공백 제거
        ingredients = ingredients.replace("\n", ", ") # 줄바꿈을 쉼표로 변환
        recipe_ingredients.append(ingredients)
        
# 정리된 재료 리스트 출력  (상위 5개만)
for i, ing in enumerate(recipe_ingredients[:5]) :
    print(f"레시피 {i+1} : {ing}")
    
# 정리된 재료 리스트를 파일로 저장 (Food2Vec) 학습용
with open("recipe_ingredients.txt", "w", encoding="utf-8") as f:
    for ing in recipe_ingredients:
        f.write(ing + "\n")
        
print("데이터 저장 완료 ('recipe_ingredients.txt' 파일 생성)")

데이터 저장 완료 ('recipe_ingredients.txt' 파일 생성)


In [14]:
import json
import re

# JSON 파일 로드
with open("all_recipes.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    
# recipe_ingredients = []

for recipe in data:
    if "RCP_PARTS_DTLS" in recipe :
        ingredients = recipe["RCP_PARTS_DTLS"]
        
        # 1. 줄바꿈 제거 및 쉼표(,) 분리
        ingredients = ingredients.replace("\n", ",")
        
        # 2. 불필요한 단위 제거 (숫자, g, 큰술, 작은술 개, 머리 등)
        ingredients = re.sub(r"\d+[\w/()]*", "", ingredients)
        
        # 3. 공백 정리 및 중복 쉼표 제거
        ingredients = re.sub(r"\s+", " ", ingredients) # 여러개의 공백을 하나로
        ingredients = re.sub(r"\s*", ",", ingredients).strip(", ") #중복된 쉼표 제거
        
        # 4. 최종 정리된 재료 추가
        recipe_ingredients.append(ingredients)
        
# 🔹 정리된 재료 리스트 출력 (상위 5개만 확인)
for i, ing in enumerate(recipe_ingredients[:5]):
    print(f"레시피 {i+1}: {ing}")

# 🔹 정리된 재료 리스트를 파일로 저장 (Food2Vec 학습용)
if len(recipe_ingredients) > 0:
    with open("recipe_ingredients.txt", "w", encoding="utf-8") as f:
        for ing in recipe_ingredients:
            f.write(ing + "\n")
    print("✅ 데이터 저장 완료! ('recipe_ingredients.txt' 파일 생성)")
else:
    print("❌ 저장할 데이터가 없습니다. JSON 파일을 확인하세요!")

레시피 1: 새,우,두,부,계,란,찜,,,연,두,부,,,,,칵,테,일,새,우,,,,,달,걀,,,,,생,크,림,,,,,설,탕,,,,,무,염,버,터,,,,고,명,,,시,금,치
레시피 2: [,],조,선,부,추,,,,,날,콩,가,루,,,,·,양,념,장,,:,,저,염,간,장,,,,,다,진,,대,파,,,,,다,진,,마,늘,,,,,고,춧,가,루,,,,,요,리,당,,,,,참,기,름,,,,,참,깨,,약,간
레시피 3: ●,방,울,토,마,토,,소,박,이,,:,,,,방,울,토,마,토,,,,,양,파,,×,,,,부,추,,,,●,양,념,장,,:,,,,고,춧,가,루,,,,,멸,치,액,젓,,,,,다,진,,마,늘,,.,,,,매,실,액,,,,,설,탕,,,,,물,,,,,통,깨,,약,간
레시피 4: ●,오,이,무,침,,:,,,오,이,,,,,다,진,,땅,콩,,,,●,순,두,부,사,과,,소,스,,:,,,,순,두,부,,,,,사,과
레시피 5: 북,엇,국,,,북,어,채,,,,,새,우,,,,,사,과,,,,,양,파,,,,,,표,고,버,섯,,,,,물
✅ 데이터 저장 완료! ('recipe_ingredients.txt' 파일 생성)


In [ ]:
import json
import re

# 🔹 JSON 파일 로드
with open("all_recipes.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 🔹 재료 목록 추출 및 전처리
recipe_ingredients = []

for recipe in data:
    if "RCP_PARTS_DTLS" in recipe:
        ingredients = recipe["RCP_PARTS_DTLS"]

        # 🔹 1. 한글 단어가 깨지는 문제 해결 (공백 기반
